In [1]:
%matplotlib inline
# %load ../../utility/initialization.py
'''
How to Train Your Dragon: V4
Sequentially initialize FT reactive distillation model automatically
'''
# system imports
import sys
import os
import datetime
sys.path.append(os.path.abspath('..'))
sys.path.append(os.path.abspath('../..'))

import numpy as np
from matplotlib import pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

# import pickle
import dill
from copy import deepcopy

# pyomo imports
from pyomo import environ as pe
from global_sets.component import m

from stages.reactive_stage import reactive_stage_rule
from stages.condenser_stage import condenser_stage_rule
from stages.reboiler_stage import reboiler_stage_rule

from utility.display_utility import beautify, beautify_reactive, HiddenLogs, HiddenPrints, plot_distribution
from utility.model_utility import add_dual, update_dual, delete_dual, check_DOF, check_iteration
from utility.model_utility import which_MPCC, select_MPCC, augmented_objective, add_solver
from utility.time_utility import create_filename_time, log_now, log_end

2018-07-25 02:55:58 - Start Program


In [2]:
logname = create_filename_time()
log_text_dir = './log/text/opt_'+logname+'.dat'
log_figure_dir = './log/figure/opt_'+logname+'.pdf'

In [3]:
with open('./log/model/stage_20_NCP.pickle','rb') as f:
    model = dill.load(f)

In [4]:
for j in model.reactive:
    model.reactive[j].MPCC_P_pf.rho = 10000
model.reboiler.MPCC_P_pf.rho = 10000

In [5]:
opt = add_solver(pe, max_iter = 500, warm_start = True, output = True, scale = True)

In [6]:
for j in model.reactive:
    select_MPCC(model.reactive[j],'pf')
select_MPCC(model.reboiler,'pf')

> Selected MPCC: reactive[1].MPCC_P_pf
s_L:  3.361333597207621e-05
s_V:  4.335208485080155e-06

> Selected MPCC: reactive[2].MPCC_P_pf
s_L:  3.70259951466815e-05
s_V:  4.259120671146189e-06

> Selected MPCC: reactive[3].MPCC_P_pf
s_L:  4.073878300491664e-05
s_V:  4.309449439021236e-06

> Selected MPCC: reactive[4].MPCC_P_pf
s_L:  4.5722775121289914e-05
s_V:  4.3556513828163175e-06

> Selected MPCC: reactive[5].MPCC_P_pf
s_L:  5.642449338138292e-05
s_V:  4.407012457041227e-06

> Selected MPCC: reactive[6].MPCC_P_pf
s_L:  7.549951280334534e-05
s_V:  4.471516219426515e-06

> Selected MPCC: reactive[7].MPCC_P_pf
s_L:  0.00012829789189381136
s_V:  4.562874196573894e-06

> Selected MPCC: reactive[8].MPCC_P_pf
s_L:  0.00037215542479177136
s_V:  4.679252465855748e-06

> Selected MPCC: reactive[9].MPCC_P_pf
s_L:  0.0004792009308648431
s_V:  5.4812007339374725e-06

> Selected MPCC: reactive[10].MPCC_P_pf
s_L:  0.00048738746659121894
s_V:  6.360616469025384e-06

> Selected MPCC: reactive[11].MPCC

In [7]:
model.del_component(model.obj)
model.obj = augmented_objective(pe,model,expr = sum(model.reactive[j].T for j in model.TRAY_reactive), sense = pe.maximize)

------------------------------------------------------------------------------------------------------------
> Obj = maximize
> reactive[8].T + reactive[9].T + reactive[10].T + reactive[11].T + reactive[12].T + reactive[14].T + reactive[15].T + reactive[16].T + reactive[17].T + reactive[18].T - reactive[1].MPCC_P_pf.pf - reactive[2].MPCC_P_pf.pf - reactive[3].MPCC_P_pf.pf - reactive[4].MPCC_P_pf.pf - reactive[5].MPCC_P_pf.pf - reactive[6].MPCC_P_pf.pf - reactive[7].MPCC_P_pf.pf - reactive[8].MPCC_P_pf.pf - reactive[9].MPCC_P_pf.pf - reactive[10].MPCC_P_pf.pf - reactive[11].MPCC_P_pf.pf - reactive[12].MPCC_P_pf.pf - reactive[13].MPCC_P_pf.pf - reactive[14].MPCC_P_pf.pf - reactive[15].MPCC_P_pf.pf - reactive[16].MPCC_P_pf.pf - reactive[17].MPCC_P_pf.pf - reactive[18].MPCC_P_pf.pf - reactive[19].MPCC_P_pf.pf - reactive[20].MPCC_P_pf.pf - reboiler.MPCC_P_pf.pf
------------------------------------------------------------------------------------------------------------


In [8]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling=mc19
max_iter=500
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
output_file=./tmp/ipopt_output_tmp.output
linear_scaling_on_demand=no


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 500                   yes
                                 mu_init = 1e-06                 yes
                             output_file = ./tmp/ipopt_output_tmp.output  yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
     

In [9]:
# pdf = PdfPages(log_figure_dir)

In [10]:
with HiddenLogs(log_text_dir,'w'):
    print('\n>','Original 20 stage case')
    print('-'*108)
    beautify(pe,model)
    log_now()

# plot_distribution(model,pdf,'Original 20 stage case')


> Original 20 stage case
------------------------------------------------------------------------------------------------------------
Here comes the result:
Total Conversion: 81.20%
------------------------------------------------------------------------------------------------------------
stages       T      Q                                            V               L       P            W     
condenser    30.00  -134.                                        3.0267          0.6407  0.0337       2.0654

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
NON--[1]     119.3  0.000       0.000  0.000  0.000  0.000       5.7667  0.0000  0.7437  0.0000       20.000
NON--[2]     131.1  0.000       0.000  0.000  0.000  0.000       5.8697  0.0000  0.6752  0.0000       20.000
NON--[3]     140.3  0.000       0.000  0.000  0.000  0.000       5.8012  0.0000  0.6136  0.0000       20.000
NON--[4]     148.7  0.000       0.000  0.000  0.000  

# Optimization Input

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw ratio
    * PR_L: 0 - 1

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
**Objective:**
* Max gasoline production

### Parameters

In [11]:
for j in model.TRAY_reactive:
    model.reactive[j].T.setlb(200+273.15)
    model.reactive[j].T.setub(300+273.15)
    
model.condenser.PR_L.unfix()
model.condenser.PR_L.setlb(0.05)
model.condenser.PR_L.setub(0.5)

In [12]:
model.reactive[4].PR_L.unfix()
model.reactive[7].PR_L.unfix()
model.reactive[10].PR_L.unfix()

### Constraints

In [13]:
model.quality_coefficient = pe.Param(within=pe.NonNegativeReals,mutable=True,initialize=1)

In [14]:
model.product_spec_con = pe.ConstraintList()

In [15]:
model.product_spec_con.add(expr = sum(model.condenser.x[i] for i in m.PRODUCT_cnumber['naphtha']) >= \
                           model.quality_coefficient*0.75*sum(model.condenser.x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reactive[7].x[i] for i in m.PRODUCT_cnumber['gasoline']) >= \
                           model.quality_coefficient*0.75*sum(model.reactive[7].x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reactive[10].x[i] for i in m.PRODUCT_cnumber['diesel']) >= \
                           model.quality_coefficient*0.6*sum(model.reactive[10].x[i] for i in m.COMP_ORG));
model.product_spec_con.add(expr = sum(model.reboiler.x[i] for i in m.PRODUCT_cnumber['heavy']) >= \
                           model.quality_coefficient*0.85*sum(model.reboiler.x[i] for i in m.COMP_ORG));

### Objective

In [16]:
model.del_component(model.obj)
model.obj = augmented_objective(pe,model,expr = model.reactive[7].L['P'], sense = pe.maximize)

------------------------------------------------------------------------------------------------------------
> Obj = maximize
> reactive[7].L[P] - reactive[1].MPCC_P_pf.pf - reactive[2].MPCC_P_pf.pf - reactive[3].MPCC_P_pf.pf - reactive[4].MPCC_P_pf.pf - reactive[5].MPCC_P_pf.pf - reactive[6].MPCC_P_pf.pf - reactive[7].MPCC_P_pf.pf - reactive[8].MPCC_P_pf.pf - reactive[9].MPCC_P_pf.pf - reactive[10].MPCC_P_pf.pf - reactive[11].MPCC_P_pf.pf - reactive[12].MPCC_P_pf.pf - reactive[13].MPCC_P_pf.pf - reactive[14].MPCC_P_pf.pf - reactive[15].MPCC_P_pf.pf - reactive[16].MPCC_P_pf.pf - reactive[17].MPCC_P_pf.pf - reactive[18].MPCC_P_pf.pf - reactive[19].MPCC_P_pf.pf - reactive[20].MPCC_P_pf.pf - reboiler.MPCC_P_pf.pf
------------------------------------------------------------------------------------------------------------


In [17]:
# itr_count1 = 0
# meta_results = {}
# while True:
#     itr_count1 += 1
#     opt = add_solver(pe, max_iter = itr_count1, warm_start = True, output = True, scale = True)
#     model_tmp = deepcopy(model)
#     results = opt.solve(model_tmp,tee=False)
#     model_tmp.solutions.store_to(results)
#     meta_results[itr_count1] = results
#     print('itr_count',itr_count1,'obj',model.obj())
#     if results.solver.termination_condition.key == 'optimal':
#         break

In [18]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [19]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Optimized Product Side Draw')
#     print('-'*108)
#     beautify(pe,model)
#     # check_product_spec(model)
#     log_now()

# # plot_distribution(model,pdf,'Optimized Product Side Draw')

# Open up feed allocation

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * Total feed = 10 kmol/s
    
**Objective:**
* Max gasoline production

In [20]:
for j in model.reactive:
    model.reactive[j].F.unfix()
    model.reactive[j].F.setlb(0)
    model.reactive[j].F.setub(10)

In [21]:
model.total_feed_con = pe.ConstraintList()

In [22]:
model.total_feed_con.add(expr = sum(model.reactive[j].F for j in model.reactive) == 10);

In [23]:
# model.reactive[17].F.setlb(1)

In [24]:
itr_count1 = 0
meta_results = {}
while True:
    itr_count1 += 1
    opt = add_solver(pe, max_iter = itr_count1, warm_start = True, output = True, scale = True)
    model_tmp = deepcopy(model)
    results = opt.solve(model_tmp,tee=False)
    model_tmp.solutions.store_to(results)
    meta_results[itr_count1] = results
    print('itr_count',itr_count1,'obj',model_tmp.obj())
    if results.solver.termination_condition.key == 'optimal':
        break

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 1 obj 0.03897160810185961
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 2 obj 0.038967883547614414
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 3 obj 0.042026387463220474
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 4 obj 0.042025550387740127
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 5 obj 0.04736479820578871
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 6 obj 0.055037600728111924
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 7 obj 0.0566254807254594
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 8 obj 0.0585982252174874
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 9 obj 0

itr_count 40 obj 0.08795787056617034
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 41 obj 0.08796043864898727
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 42 obj 0.08796074904974616
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 43 obj 0.08796304519550213
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 44 obj 0.08796442335393405
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 45 obj 0.08797024129915305
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 46 obj 0.08797129031538375
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 47 obj 0.08797116469215846
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 48 obj 0.08796837886686482
	message from solver=Ipopt 3.12.8\x3a Maximum Numbe

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 80 obj 0.0894481926787917
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 81 obj 0.09108958632513922
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 82 obj 0.0920123329000765
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 83 obj 0.0942680049021603
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 84 obj 0.0977997172768017
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 85 obj 0.10248450757486849
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 86 obj 0.10375289912754584
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 87 obj 0.10377835176089828
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 88 ob

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 119 obj 0.11440877392084124
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 120 obj 0.11541776431279246
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 121 obj 0.11620466530243796
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 122 obj 0.11578065750063717
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 123 obj 0.1158037760445004
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 124 obj 0.11581092860686228
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 125 obj 0.11581275640781326
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 126 obj 0.11582116755117954
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_

	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 158 obj 0.11603566262020007
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 159 obj 0.11603608719108081
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 160 obj 0.11604369337181826
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 161 obj 0.11604402446090524
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 162 obj 0.11604869114043219
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 163 obj 0.11606744801651955
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 164 obj 0.11607307914898639
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr_count 165 obj 0.11607667788783127
	message from solver=Ipopt 3.12.8\x3a Maximum Number of Iterations Exceeded.
itr

In [25]:
with open('./meta_results_1step_uncontrolled.pickle','wb') as f:
    dill.dump(meta_results,f)

In [26]:
results = opt.solve(model,tee=True)
update_dual(pe,model)

Ipopt 3.12.8: print_user_options=yes
linear_solver=ma86
linear_system_scaling=mc19
max_iter=186
warm_start_init_point=yes
warm_start_bound_push=1e-20
warm_start_mult_bound_push=1e-20
mu_init=1e-06
output_file=./tmp/ipopt_output_tmp.output
linear_scaling_on_demand=no


List of user-set options:

                                    Name   Value                used
                linear_scaling_on_demand = no                    yes
                           linear_solver = ma86                  yes
                   linear_system_scaling = mc19                  yes
                                max_iter = 186                   yes
                                 mu_init = 1e-06                 yes
                             output_file = ./tmp/ipopt_output_tmp.output  yes
                      print_user_options = yes                   yes
                   warm_start_bound_push = 1e-20                 yes
                   warm_start_init_point = yes                   yes
     

  58 -8.7665064e-02 2.49e-04 4.85e-02  -6.0 2.07e-01  -4.1 1.00e+00 1.00e+00h  1
  59 -8.7753462e-02 1.73e-03 4.25e-01  -6.0 6.34e-01  -4.6 1.00e+00 1.00e+00h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
  60 -8.7870041e-02 3.72e-03 3.34e+00  -6.0 1.87e+00  -5.1 1.00e+00 5.49e-01h  1
  61 -8.7910325e-02 3.80e-03 2.44e+01  -6.0 4.59e+00  -5.5 1.00e+00 1.20e-01f  1
  62 -8.8111483e-02 1.05e-01 1.56e+01  -6.0 6.58e+00  -6.0 1.00e+00 6.91e-01f  1
  63 -8.8442985e-02 2.56e-01 7.24e+01  -6.0 9.33e+00  -6.5 1.00e+00 8.94e-01h  1
  64 -8.8439067e-02 2.40e-01 6.81e+01  -6.0 3.16e+00  -2.5 4.98e-02 6.03e-02h  1
  65 -8.8438070e-02 2.32e-01 5.74e+01  -6.0 3.05e+00  -2.9 2.94e-01 3.49e-02h  1
  66 -8.8437054e-02 2.29e-01 5.67e+01  -6.0 2.18e+00  -1.6 4.83e-03 1.29e-02h  1
  67 -8.8435184e-02 2.27e-01 5.60e+01  -6.0 6.11e+00  -2.1 5.38e-03 1.03e-02h  1
  68 -8.8432892e-02 2.20e-01 5.39e+01  -6.0 1.89e+00  -1.6 1.25e-01 2.92e-02f  1
  69 -8.8406813e-02 1.65e-01

 151 -1.1592488e-01 1.30e-05 2.00e-06  -6.0 1.06e+00  -7.6 1.00e+00 1.00e+00h  1
 152 -1.1599497e-01 1.05e-02 6.74e+01  -9.0 2.69e+01  -8.1 7.48e-01 5.22e-01h  1
 153 -1.1601685e-01 4.31e-03 4.85e+01  -9.0 9.70e-01  -5.8 8.76e-01 5.95e-01h  1
 154 -1.1603084e-01 2.30e-03 2.52e+01  -9.0 2.12e+00  -6.3 4.43e-01 4.75e-01h  1
 155 -1.1603246e-01 2.04e-03 2.23e+01  -9.0 2.48e+00  -5.9 1.15e-01 1.13e-01h  1
 156 -1.1603344e-01 1.84e-03 2.05e+01  -9.0 8.64e-01  -5.5 1.88e-01 1.02e-01h  1
 157 -1.1603482e-01 1.55e-03 1.66e+01  -9.0 1.29e+00  -5.9 1.72e-03 1.58e-01f  1
 158 -1.1603568e-01 1.47e-03 1.57e+01  -9.0 2.58e+00  -6.4 2.06e-05 4.60e-02f  1
 159 -1.1603612e-01 1.46e-03 2.90e+02  -9.0 5.79e+00  -6.9 5.91e-01 1.08e-02h  1
iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
 160 -1.1604372e-01 1.13e-03 9.52e+01  -9.0 7.33e+00  -7.4 6.09e-01 2.52e-01f  1
 161 -1.1604405e-01 1.12e-03 9.41e+01  -9.0 1.96e+01  -7.8 1.10e-02 1.42e-02h  1
 162 -1.1604872e-01 1.18e-03

In [27]:
with HiddenLogs(log_text_dir):
    print('\n>','Optimized Feed Amount and Location')
    print('-'*108)
    beautify(pe,model)
    # check_product_spec(model)
    log_now()

# plot_distribution(model,pdf,'Optimized Feed Amount and Location')


> Optimized Feed Amount and Location
------------------------------------------------------------------------------------------------------------
Here comes the result:
Total Conversion: 86.08%
------------------------------------------------------------------------------------------------------------
stages       T      Q                                            V               L       P            W     
condenser    30.00  -140.                                        2.3281          0.6306  0.0877       2.2018

stages       T      Q           r_FT   Conv%  F      cat         V       Re      L       P            P_VLE 
NON--[1]     123.0  0.000       0.000  0.000  0.000  0.000       5.2483  0.0000  0.7493  0.0000       20.000
NON--[2]     133.4  0.000       0.000  0.000  0.000  0.000       5.3670  0.0000  0.6985  0.0000       20.000
NON--[3]     140.9  0.000       0.000  0.000  0.000  0.000       5.3161  0.0000  0.6581  0.0000       20.000
NON--[4]     147.4  0.000       0.000  0.

# Increase spec a little bit

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= **0.8**
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * Total feed = 10 kmol/s
    
**Objective:**
* Max gasoline production

In [28]:
# model.product_spec_con.add(expr = sum(model.reactive[7].x[i] for i in product['gasoline']) >= \
#                            model.quality_coefficient*0.8*sum(model.reactive[7].x[i] for i in m.COMP_ORG));

In [29]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [30]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Increase Gasoline Specification to 80%')
#     print('-'*108)
#     beautify(pe,model)
#     check_product_spec(model)
#     log_now()

# plot_distribution(model,pdf,'Increase Gasoline Specification to 80%')

# Open up catalyst allocation ( = TOTAL reconstruction, long wait time)

**Parameters:**
* Stage Temperature
    * Rectifying section: 200C - 300C
    * Stripping section: 200C - 300C
* Reflux
    * Distillate / (Distillate + Reflux): 0.5 - 0.05, Refulx Ratio: 1 - 19
* Side-draw
    * PR_L: 0 - 1
    
* Feed
    * F: 0 - 3
        
* Catalyst
    * cat: 0 - 10000

**Constraints:**
* Product
    * Distillate: C5~C7 >= 0.75
    * Gasoline: C8~C12 >= 0.75 / 0.8
    * Diesel: C13~C18 >= 0.6
    * Heavy: C19+ >= 0.85
    
* Total feed
    * sum feed = 10 kmol/s
    
* Total catalyst
    * sum cat = 30000 kg
    
**Objective:**
* Max gasoline production

In [31]:
# for j in model.reactive:
#     model.reactive[j].cat.unfix()
#     model.reactive[j].cat.setlb(0)
#     model.reactive[j].cat.setub(30000)

In [32]:
# model.total_cat_con = pe.ConstraintList()

In [33]:
# model.total_cat_con.add(expr = sum(model.reactive[j].cat for j in model.reactive) == 10*3000);

In [34]:
# results = opt.solve(model,tee=True)
# update_dual(pe,model)

In [35]:
# with HiddenLogs(log_text_dir):
#     print('\n>','Optimized Catalyst Amount and Location')
#     print('-'*108)
#     beautify(pe,model)
#     check_product_spec(model)
#     log_now()

# plot_distribution(model,pdf,'Optimized Catalyst Amount and Location')

In [36]:
# pdf.close()

In [37]:
# with open('./log/model/{}.pickle'.format(logname),'wb') as f:
#     dill.dump(model,f)

In [38]:
# for j in model.TRAY_reactive:
#     print('stage',j,'\n')
#     print('Total Flow\t{}'.format(model.reactive[j].F.value + sum(model.reactive[j].r_total_comp[i].value for i in m.COMP_TOTAL)))
#     for i in m.COMP_TOTAL:
#         if i in m.COMP_FEED:
#             print('{}\t\t{}'.format(i,model.reactive[j].F.value*model.reactive[j].z[i].value + model.reactive[j].r_total_comp[i].value))
#         else:
#             print('{}\t\t{}'.format(i,model.reactive[j].r_total_comp[i].value))